## Agenda
- Extract the data from spotify api (my_playlist)

> note: this playlist will update every week.

External lib <br>
- pip install spotipy
- pip install pandas

In [2]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import os

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id="os.environ['client_id']", client_secret=os.environ['client_secret'])

In [7]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [8]:
playlist_link='https://open.spotify.com/playlist/XXXXXXXXXXXXXXXXXXXX'

In [9]:
playlist_URI=playlist_link.split("/")[-1]
print(playlist_URI)

6iPkekWWYMBhLCFUDyliS8


In [10]:
data = sp.playlist_tracks(playlist_URI)

print(data)

{'href': 'https://api.spotify.com/v1/playlists/6iPkekWWYMBhLCFUDyliS8/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_at': '2025-01-05T23:45:15Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/31rfk2k7piikvhbd3z5v77jocmjm'}, 'href': 'https://api.spotify.com/v1/users/31rfk2k7piikvhbd3z5v77jocmjm', 'id': '31rfk2k7piikvhbd3z5v77jocmjm', 'type': 'user', 'uri': 'spotify:user:31rfk2k7piikvhbd3z5v77jocmjm'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': None, 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'L

## Convert data to JSON format

In [11]:
# Convert data to JSON format
json_data = json.dumps(data)

# Save JSON data to a file
with open('data.json', 'w') as f:
    f.write(json_data)

In [12]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                        'total_tracks':album_total_tracks,'url':album_url}
    album_list.append(album_element)

In [13]:
artist_list = []
for row in data['items']:
    for key, value in row.items():
        if key == "track":
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'], 'artist_name':artist['name'], 'external_url': artist['href']}
                artist_list.append(artist_dict)

In [14]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_element = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                    'popularity':song_popularity,'song_added':song_added,'album_id':album_id,
                    'artist_id':artist_id
                   }
    song_list.append(song_element)

## Album table data frame

In [15]:
album_df = pd.DataFrame.from_dict(album_list)
# display(album_df)

In [16]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      9 non-null      object
 1   name          9 non-null      object
 2   release_date  9 non-null      object
 3   total_tracks  9 non-null      int64 
 4   url           9 non-null      object
dtypes: int64(1), object(4)
memory usage: 492.0+ bytes


### Remove duplicates

In [17]:
album_df = album_df.drop_duplicates(subset=['album_id'])
display(album_df)

,album_id,name,release_date,total_tracks,url
0,1DzYor2sqKFs4Z6qcUl0V5,Jhoomo Re,2011-06-03,6,https://open.spotify.com/album/1DzYor2sqKFs4Z6...
1,3I3kZyHUtEA9Y59rJkxtk6,"Sajni (From ""Laapataa Ladies"")",2024-02-12,1,https://open.spotify.com/album/3I3kZyHUtEA9Y59...
2,5ZbqiWukRFxq8zwllvK3YT,Sahiba,2024-11-15,1,https://open.spotify.com/album/5ZbqiWukRFxq8zw...
3,3oFkgvHruat7gfJMAJv6Wa,Trending Punjabi Hits,2024-07-25,20,https://open.spotify.com/album/3oFkgvHruat7gfJ...
4,1RNEvKlb3YovuiE8Gw5VOO,Laila Majnu (Original Motion Picture Soundtrack),2018-10-30,11,https://open.spotify.com/album/1RNEvKlb3YovuiE...
5,0upenH0uUT36nBbVM5mQhW,Jo Tum Mere Ho,2024-08-02,1,https://open.spotify.com/album/0upenH0uUT36nBb...
6,2SarxpD48O1DOiDIccyZ0W,Husn,2023-12-01,1,https://open.spotify.com/album/2SarxpD48O1DOiD...
7,0wkGNaTXbbxaD9QjvxlNGE,G.O.A.T.,2020-07-29,16,https://open.spotify.com/album/0wkGNaTXbbxaD9Q...


In [18]:
album_df['release_date'] = pd.to_datetime(album_df['release_date'])

In [19]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   album_id      8 non-null      object        
 1   name          8 non-null      object        
 2   release_date  8 non-null      datetime64[ns]
 3   total_tracks  8 non-null      int64         
 4   url           8 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 384.0+ bytes


## Artist DF

In [20]:
artist_df = pd.DataFrame.from_dict(artist_list)

In [21]:
artist_df = artist_df.drop_duplicates(subset=['artist_id'])

In [22]:
artist_df.head()

,artist_id,artist_name,external_url
0,4oVMLzAqW6qhRpZWt8fNw4,Kailash Kher,https://api.spotify.com/v1/artists/4oVMLzAqW6q...
1,3PgeUFj9oHU4K4RGaITz2f,Paresh Kamath,https://api.spotify.com/v1/artists/3PgeUFj9oHU...
2,79sho4jpEGoCCC51KHnNSJ,Naresh Kamath,https://api.spotify.com/v1/artists/79sho4jpEGo...
3,3PpjqSKBer3obW6lDflOBx,Ram Sampath,https://api.spotify.com/v1/artists/3PpjqSKBer3...
4,4YRxDV8wJFPHPTeXepOstw,Arijit Singh,https://api.spotify.com/v1/artists/4YRxDV8wJFP...


## Song Dataframe

In [23]:
song_df = pd.DataFrame.from_dict(song_list)

In [24]:
song_df.head()

,song_id,song_name,duration_ms,url,popularity,song_added,album_id,artist_id
0,2vR23BjIzpLMtzsyKrM6Mb,Saiyyan,343906,https://open.spotify.com/track/2vR23BjIzpLMtzs...,70,2025-01-05T23:45:15Z,1DzYor2sqKFs4Z6qcUl0V5,7nEKS1t7i0pE29k2EiMSW6
1,5zCnGtCl5Ac5zlFHXaZmhy,"Sajni (From ""Laapataa Ladies"")",170044,https://open.spotify.com/track/5zCnGtCl5Ac5zlF...,79,2025-01-05T23:45:21Z,3I3kZyHUtEA9Y59rJkxtk6,3PpjqSKBer3obW6lDflOBx
2,4ulroyDK6rN73nVkCkWhur,Sahiba,217148,https://open.spotify.com/track/4ulroyDK6rN73nV...,78,2025-01-05T23:45:39Z,5ZbqiWukRFxq8zwllvK3YT,74OaRjmyh0XyRZsQQQ5l7c
3,2ruImJNc00PYkuDumhV98V,Jannat,220500,https://open.spotify.com/track/2ruImJNc00PYkuD...,31,2025-01-05T23:46:23Z,3oFkgvHruat7gfJMAJv6Wa,0LyfQWJT6nXafLPZqxe9Of
4,5sUSbJo0brScvv48ZZ1HrQ,O Meri Laila,281856,https://open.spotify.com/track/5sUSbJo0brScvv4...,73,2025-01-05T23:46:54Z,1RNEvKlb3YovuiE8Gw5VOO,3Qol2r2jehhR5yKbmk1Dg4


In [25]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   song_id      9 non-null      object
 1   song_name    9 non-null      object
 2   duration_ms  9 non-null      int64 
 3   url          9 non-null      object
 4   popularity   9 non-null      int64 
 5   song_added   9 non-null      object
 6   album_id     9 non-null      object
 7   artist_id    9 non-null      object
dtypes: int64(2), object(6)
memory usage: 708.0+ bytes


In [26]:
song_df['song_added'] =  pd.to_datetime(song_df['song_added'])

In [27]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   song_id      9 non-null      object             
 1   song_name    9 non-null      object             
 2   duration_ms  9 non-null      int64              
 3   url          9 non-null      object             
 4   popularity   9 non-null      int64              
 5   song_added   9 non-null      datetime64[ns, UTC]
 6   album_id     9 non-null      object             
 7   artist_id    9 non-null      object             
dtypes: datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 708.0+ bytes
